# An accurate boundary-integral formulation for the calculation of electrostatic forces with an implicit-solvent model

### Import libraries

In [1]:
import os
import numpy as np
import pbj  # 

### Import custom-functions 

In [2]:
#magnitud of a vector numpy
def mag(v):
    return np.sqrt(np.dot(v, v))

#Translation pqr
def pqr_translate(directory, protein_pqr, distance):
    'Translate along distance vector the protein .pqr file'

    protein_pqr_d = protein_pqr + '_d' + str(distance[2]) #new name with z-distance
    mesh_pqr_path = '{}/{}.pqr'.format(directory,protein_pqr)
    mesh_pqr_path_out = '{}/{}.pqr'.format(directory,protein_pqr_d)

    pqr_file = open(mesh_pqr_path, 'r')
    pqr_data = pqr_file.read().split('\n')
    pqr_t_file = open(mesh_pqr_path_out, 'w')
    for line in pqr_data:
        line = line.split()
        if len(line) == 0 or line[0] != 'ATOM':
            continue
        tx,ty,tz = distance
        x_new = float(line[5])+ float(tx)
        y_new = float(line[6])+ float(ty)
        z_new = float(line[7])+ float(tz)
        text_template = '{} {}  {}   {} {}        {: 1.3f}  {: 1.3f}  {: 1.3f}  {}  {} \n'
        pqr_t_file.write(text_template.format(line[0],line[1],line[2],line[3],line[4],x_new,y_new,z_new,line[8],line[9]))

    pqr_file.close() 
    pqr_t_file.close()

    return None

In [20]:
pqr_translate('/home/ianaddisonsmith/pbj/notebooks/tutorials/pqrs','1brs_BE_barstar',[0,0,1000])

## Two centered-charge (2q) unit spheres

### Simulation at several distances between centers 

In [3]:
dist_sim = [3]
for dist in dist_sim:
    print('-----------------------------------')
    print('Distance: %d A'%(dist))

    #Solvation energy alone
    simulation = pbj.electrostatics.Simulation(formulation='direct')
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=16, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.calculate_solvation_energy()
    solv_energy_alone = protein.results['solvation_energy']

    #Solvation energy complex
    simulation = pbj.electrostatics.Simulation(formulation='direct')
    protein = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2.pqr", mesh_density=16, mesh_generator="msms")
    protein2 = pbj.electrostatics.Solute("pqrs//sphere_point_r1q2_d%d.pqr"%(dist), mesh_density=16, mesh_generator="msms")
    simulation.add_solute(protein)
    simulation.add_solute(protein2)
    simulation.calculate_solvation_energy()
    solv_energy_complex = protein.results['solvation_energy'] + protein2.results['solvation_energy']
    solv_energy_binding = solv_energy_complex - 2*solv_energy_alone

    #Forces calculation   
    print('Dielectric approx formulation')
    simulation.calculate_solvation_forces(force_formulation='energy_functional', fdb_approx=True)
    f_solv_full_app = protein2.results['f_solv']
    fdb_app = protein2.results['f_db']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Dielectric exact formulation')
    simulation.calculate_solvation_forces(force_formulation='energy_functional', fdb_approx=False)
    f_solv_full_ex = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

    print('Maxwell tensor formulation')
    simulation.calculate_solvation_forces(force_formulation='maxwell_tensor')
    f_solv_fast = protein2.results['f_solv']
    print(protein.results['f_solv'])
    print(protein2.results['f_solv'])

-----------------------------------
Distance: 3 A
C:\Users\ianad\Desktop\paper_PBforces\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2.xyzr -of c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2 -p 1.4 -d 16 -no_header
C:\Users\ianad\Desktop\paper_PBforces\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2.xyzr -of c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2 -p 1.4 -d 16 -no_header
C:\Users\ianad\Desktop\paper_PBforces\pbj\mesh\ExternalSoftware\MSMS\msms.exe -if c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2_d3.xyzr -of c:\Users\ianad\Desktop\paper_PBforces\mesh_temp\sphere_point_r1q2_d3 -p 1.4 -d 16 -no_header
Dielectric approx formulation
[-1.86193785e+00  3.41152215e-02  1.42122382e-04]
[1.86193705e+00 3.41152965e-02 1.42272985e-04]
Dielectric exact formulation
[-1.94353017e+00  3.45402637e-02  1.51437632e-0

In [4]:
from bempp.api.assembly.blocked_operator import (
        grid_function_list_from_coefficients,
    )
phi2 = grid_function_list_from_coefficients(
        (protein.results['phi'].evaluate_on_element_centers()[0]**2)*protein.mesh.volumes.ravel(), [protein.matrices['A'].domain_spaces[0]])
d_phi2 = grid_function_list_from_coefficients(
        (protein.results['d_phi'].evaluate_on_element_centers()[0]**2)*protein.mesh.volumes.ravel(), [protein.matrices['A'].domain_spaces[1]])
P_normal_mag = grid_function_list_from_coefficients(
        protein.results['P_normal'], [protein.matrices['A'].domain_spaces[1]])[0]

import bempp.api
bempp.api.PLOT_BACKEND = "gmsh"
to_kcalmolA = 4*np.pi*332.0636817823836
f_db = to_kcalmolA*-0.5*(protein.ep_ex-protein.ep_in)*(protein.ep_in/protein.ep_ex)*d_phi2[0]
f_ib = to_kcalmolA*-0.5*protein.kappa*protein.ep_ex*phi2[0]

In [24]:
bempp.api.export('msh/P_normal.msh', grid_function=P_normal_mag,data_type='node')
bempp.api.export('msh/f_db.msh', grid_function=f_db,data_type='node')
bempp.api.export('msh/f_ib.msh', grid_function=f_ib,data_type='node')
bempp.api.export('msh/phi.msh', grid_function=protein.results['phi'],data_type='node')
bempp.api.export('msh/d_phi.msh', grid_function=protein.results['d_phi'],data_type='node')